In [25]:
import os
import cv2
import numpy as np
from mtcnn import MTCNN
from keras_facenet import FaceNet
from sklearn.preprocessing import Normalizer, LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import pickle

In [26]:
#Instantiate FaceNet wrapper
embedder = FaceNet()  # automatically downloads & loads the model
facenet_model = embedder.model  # underlying Keras model
print(f"FaceNet loaded; input shape: {facenet_model.input_shape}")

FaceNet loaded; input shape: (None, None, None, 3)


In [27]:
detector = MTCNN()

def extract_face_from_frame(frame, required_size=(160, 160)):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = detector.detect_faces(image)
    if not results:
        return None
    x1, y1, w, h = results[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + w, y1 + h
    face = image[y1:y2, x1:x2]
    face = cv2.resize(face, required_size)
    return face

In [ ]:
# Embedder and dataset functions
def get_embedding(face_pixels):
    # use wrapper's preprocess and embed methods
    emb = embedder.embeddings([face_pixels])
    return emb[0]

# Load existing dataset images and compute embeddings
def load_dataset_embeddings(dataset_path='dataset/'):
    faces, names = [], []
    for person in os.listdir(dataset_path):
        person_dir = os.path.join(dataset_path, person)
        for img_name in os.listdir(person_dir):
            img_path = os.path.join(person_dir, img_name)
            img = cv2.imread(img_path)
            face = extract_face_from_frame(img)
            if face is None: continue
            faces.append(face)
            names.append(person)
    embeddings = [get_embedding(f) for f in faces]
    return np.asarray(embeddings), np.asarray(names)

In [29]:
from sklearn.exceptions import NotFittedError

def train_classifier(embeddings, names):
    # Expecting embeddings shape (num_samples, feat_dim)
    if embeddings.size == 0 or names.size == 0:
        raise ValueError("No data available for training. Add some face images first.")
    # Normalize embeddings
    in_enc = Normalizer(norm='l2')
    emb_norm = in_enc.transform(embeddings)
    # Encode labels
    out_enc = LabelEncoder()
    y = out_enc.fit_transform(names)
    # If only one class, use KNN to avoid SVM errors
    from sklearn.neighbors import KNeighborsClassifier
    if len(out_enc.classes_) == 1:
        print(f"Only one identity ('{out_enc.classes_[0]}') found. Using KNN classifier (k=1) to handle single class.")
        model = KNeighborsClassifier(n_neighbors=1)
    else:
        # Train SVM for multi-class
        model = SVC(kernel='linear', probability=True)
    model.fit(emb_norm, y)
    # Save artifacts
    with open('svc_classifier.pkl', 'wb') as f:
        pickle.dump(model, f)
    with open('in_encoder.pkl', 'wb') as f:
        pickle.dump(in_enc, f)
    with open('out_encoder.pkl', 'wb') as f:
        pickle.dump(out_enc, f)
    print('Training complete; classifier and encoders saved.')
    return model, in_enc, out_enc

# Initial train or load
try:
    if os.path.exists('svc_classifier.pkl'):
        # Load existing classifier
        with open('svc_classifier.pkl', 'rb') as f:
            svc_model = pickle.load(f)
        with open('in_encoder.pkl', 'rb') as f:
            in_encoder = pickle.load(f)
        with open('out_encoder.pkl', 'rb') as f:
            out_encoder = pickle.load(f)
        print('Loaded existing classifier and encoders.')
    else:
        # No existing model: attempt to train from dataset
        print('No existing classifier found — attempting initial training from dataset...')
        emb, names = load_dataset_embeddings()
        if names.size == 0:
            print("[Warning] Dataset folder is empty. Please add a new person first via the interactive cell.")
        else:
            svc_model, in_encoder, out_encoder = train_classifier(emb, names)
except (ValueError, NotFittedError) as e:
    print(f"Error during training/loading: {e}")

Loaded existing classifier and encoders.


In [30]:
#Add new identity or recognize existing
choice = input("Is this a new person? (yes/no): ").strip().lower()
if choice == 'yes':
    name = input("Enter new person name: ").strip()
    save_dir = os.path.join('dataset', name)
    os.makedirs(save_dir, exist_ok=True)
    cam = cv2.VideoCapture(0)
    count = 0
    cv2.namedWindow('Capture')
    print("Press SPACE to capture 3 face images.")
    while count < 3:
        ret, frame = cam.read()
        if not ret:
            continue
        cv2.imshow('Capture', frame)
        key = cv2.waitKey(1)
        if key == ord(' '):  # SPACE key
            face = extract_face_from_frame(frame)
            if face is not None:
                img_bgr = cv2.cvtColor(face, cv2.COLOR_RGB2BGR)
                cv2.imwrite(os.path.join(save_dir, f'{count+1}.jpg'), img_bgr)
                count += 1
                print(f"Saved image {count}/3")
    cam.release()
    cv2.destroyWindow('Capture')
    # Retrain classifier with new data
    emb, names = load_dataset_embeddings()
    svc_model, in_encoder, out_encoder = train_classifier(emb, names)
    print(f"Added '{name}' and retrained model.")

# Recognition branch (no extra prompts)
cam = cv2.VideoCapture(0)
cv2.namedWindow('Recognize')
print("Press SPACE to capture image for recognition.")
while True:
    ret, frame = cam.read()
    if not ret:
        continue
    cv2.imshow('Recognize', frame)
    key = cv2.waitKey(1)
    if key == ord(' '):  # SPACE key
        face = extract_face_from_frame(frame)
        break
cam.release()
cv2.destroyWindow('Recognize')

# Prediction
emb_vec = get_embedding(face)
emb_norm = in_encoder.transform([emb_vec])
pred = svc_model.predict(emb_norm)
print(f"Recognized as: {out_encoder.inverse_transform(pred)[0]}")

Press SPACE to capture image for recognition.
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Recognized as: Sayan2
